# **Procesamiento del habla**

## Estructura de sub-etapas

---

```
├── Input Layer
│   ├── Audio Capture
│   └── ASR (Speech-to-Text)
├── NLP Layer
│   ├── Text Preprocessing
│   ├── Intent Detection
│   ├── Entity Recognition
│   └── Sentiment Analysis
├── Dialogue Management
│   ├── State Management
│   ├── Response Generation
│   ├── Response Selection
│   └── Context Handling
├── Output Layer
│   ├── TTS (Text-to-Speech)
│   └── Audio Playback

```


*Bugs*
- [ ] Comentarios en español y aclarativos

- [ ] Completar estructura operativa

- [ ] Limpiar librerias y paquetes dejando solo los que se usen

- [ ] Esqueleto asci de etapas: 

    depurado

    en español

- [ ] Estructura markdown

    Jerarquia de encabezados

    Negritas

    Colores resaltados

- [ ] hablar con francisco sobre dic de salida y como resivo esos datos
- [ ] Definir como se conectara esta etapa con despliegue:

- [ ] agregar id_consulta a archivos de entrada y salida

    
    llamara a la funcion chatbot?
    
    exporto un diccionario
    
    Exporto un modelo?

- [ ] text to speech

    Corregir pronunciacion ars

- [ ] reconocimiento de voz

#### _Estructura operativa para cada sub-etapa:_

**Selección de técnicas de modelado:** En este caso se utilizara [] para la etapa de reconocimiento de voz, [] para el procesamiento y analisis de texto y [] para la conversion de texto a voz.

**Generación de un diseño de comprobación:** Para elegir el modelo correcto, este devera ser el que pondere mas alto en un promedio de las diferentes metricas de evaluacion del modelo.

**Generación de los modelos:** Se definiran y configuraran los parametros del modelo para pasar a la etapa de ejecucion y descripcion.

**Evaluación del modelo:** Se evaluaran los diferentes modelos de manera individual en busqueda de optimizar sus parametros y escoger la combinacion mas optima entre modelos / parametros individuales.

### Librerias y paquetes

#### *Entorno virtuales*

In [ ]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

#### *Instalacion*

In [ ]:
# Librerias de preprocesamiento
!pip3 install sounddevice
!pip3 install scipy
!pip3 install pydub

# Librerias de reconocimiento de voz
!pip3 install pocketsphinx
!pip3 install SpeechRecognition

# Librerias de generacion de voz
!pip3 install gtts
# Librerias de analisis linguistico
!pip3 install spacy

In [ ]:
!python3 -m spacy download es_core_news_sm

#### *Importacion*

In [ ]:
# Librerías para reconocimiento de voz y procesamiento de audio
import speech_recognition as sr
import sounddevice as sd
from scipy.io.wavfile import write
import queue
import threading
from pydub import AudioSegment

# Librerías para manipulación de archivos y compresión
import wave
import zipfile
import os
import subprocess

# Librerías para procesamiento de datos y análisis
import numpy as np
import pandas as pd
import spacy
import re

# Librería para generación de voz
from gtts import gTTS

# Importación de funciones comunes a otros cuadernos
from funciones_comunes import common_functions

## 1. Reconocimiento del Habla

___

In [ ]:
# Ruta del archivo zip descargado
ruta_modelo_comprimido_pocketsphinx_esp = "../datos/brutos/modelos_proc_habla/modelo_esp.zip"

# Directorio donde se colocarán los archivos descomprimidos
ruta_modelo_descomprimido_pocketsphinx_esp = "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/"

# Descomprimir el archivo zip
with zipfile.ZipFile(ruta_modelo_comprimido_pocketsphinx_esp, 'r') as zip_ref:
    zip_ref.extractall(ruta_modelo_descomprimido_pocketsphinx_esp)

print("Archivos descomprimidos en:", ruta_modelo_descomprimido_pocketsphinx_esp)

In [ ]:
try:
    subprocess.run(["sudo", "apt-get", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en sistemas basados en Debian.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)
try:
    subprocess.run(["brew", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en macOS utilizando Homebrew.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)

### DataFrames

In [ ]:
consulta_dict = {}

In [ ]:
path_csv_rutas_verduras = '../datos/procesaodos/VerdurasporSupermercado.csv'
df_frutas_verduras = pd.read_csv(path_csv_rutas_verduras)

In [ ]:
df_frutas_verduras.head(10)

### a. Captura de entradas de texto

In [ ]:
#Conexion con api de fast-api
# aui iria la conexion con un script encargado de conectar con la api de fast-api

In [ ]:
texto_prueba = 'estoy buscando tomates en oferta pero que esten bien rojitos :p'

### b. Captura de Audio

In [ ]:
SAMPLE_RATE = 44100  # Tasa de muestreo
DURATION = 10  # Duración de la grabación en segundos
AUDIO_FILES_PATH = "../datos/brutos/audios_proc_habla/"
FILENAME = "output"  # Nombre del archivo de salida
EXTENCION_ENTRADA = '.wav'
EXTENCION_SALIDA = '.aiff'

In [ ]:
ruta_audio_entrada = AUDIO_FILES_PATH + FILENAME + EXTENCION_ENTRADA
ruta_audio_entrada_convertido = AUDIO_FILES_PATH + 'waw_conv_' + FILENAME + EXTENCION_SALIDA

In [ ]:
def record_audio(audio_file_path = AUDIO_FILES_PATH, filename = FILENAME, duration = DURATION, sample_rate = SAMPLE_RATE):
    print("Grabando...")
    # Grabar audio con 1 canal (mono)
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Esperar a que termine la grabación
    print("Grabación finalizada")
    write(ruta_audio_entrada, sample_rate, recording)
    return filename

In [ ]:
def convert_to_wav(input_file = ruta_audio_entrada, output_file = ruta_audio_entrada_convertido):
    sound = AudioSegment.from_file(input_file)
    sound.export(output_file, format="aiff")

In [ ]:
def chatbot():
    record_audio()

In [ ]:
chatbot()

In [ ]:
# Uso de la función convert_to_wav
convert_to_wav(ruta_audio_entrada, ruta_audio_entrada_convertido)

#### Reconocimiento del Habla (ASR - Automatic Speech Recognition):

In [ ]:
# Crear un objeto Recognizer
recognizer = sr.Recognizer()

In [ ]:
def recognize_speech():
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(ruta_audio_entrada_convertido) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_sphinx(audio_data, language="es-ES")
            print("Texto reconocido:", text)
    except sr.UnknownValueError:
        print("No se pudo entender el audio")
    except sr.RequestError as e:
        print("Error al solicitar resultados de reconocimiento de voz; {0}".format(e))

In [ ]:
# Ejecutar la función para reconocer el discurso
recognize_speech()

In [ ]:
# Abrir el archivo de audio
def transcribe_audio(audio_lang = 'es-ES'):
    with sr.AudioFile(ruta_audio_entrada_convertido) as source:
        # Escuchar el audio (en inglés)
        audio = recognizer.record(source)

        # Utilizar Google Speech Recognition para transcribir el audio
        try:
            text = recognizer.recognize_google(audio, audio_lang)
            print("Transcripción: ", text)
            return text
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
        except sr.RequestError as e:
            print("Error al solicitar resultados del servicio Google Speech Recognition; {0}".format(e))

In [ ]:
transcribe_audio()

In [ ]:
# Transcribir y mostrar el resultado
text = transcribe_audio(audio_file_path, audio_lang)

## 2. Procesamiento del Lenguaje Natural

___

In [ ]:
# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

### Preprocesamiento de texto


In [ ]:
# Lista de abreviaturas y formas no abreviadas de unidades de medida de peso
unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',
                        'gramos', 'kilogramos', 'miligramos', 'microgramos',
                        'toneladas', 'libras', 'onzas', 'quintales']

In [ ]:
def procesar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[,\.;:!\-*#@$!+_%^&`~]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    thresgold = 1

    doc = nlp(texto)

    tokens = [token.text for token in doc]
    stopwords = spacy.lang.es.stop_words.STOP_WORDS
    filtered_tokens = [
        token.text for token in doc
        if token.text not in stopwords
        and token.pos_ in ('NOUN', 'ADJ', 'ADP')
        and len(token.text) > thresgold
        and token.text not in unidades_medida_peso]
    lemmas = [token.lemma_ for token in doc if token.text in filtered_tokens]
    pos_tags = [(token.text, token.pos_) for token in doc if token.text in filtered_tokens]

    return {
        "tokens": tokens,
        "lemmas": lemmas,
        "filtered_tokens": filtered_tokens,
        "pos_tags": pos_tags,
    }


**Normalización del texto (eliminación de ruido, corrección ortográfica, etc.)**

In [ ]:
texto_analizado = procesar_texto(texto_prueba)

**Tokenización**

In [ ]:
tokens = texto_analizado['tokens']

In [ ]:
tokens

**Eliminación de stop words.**


In [ ]:
tokens_filtrados = texto_analizado['filtered_tokens']

In [ ]:
tokens_filtrados

**Lematización y stemming**

In [ ]:
lemmas_filtrados = texto_analizado['lemmas']

In [ ]:
lemmas_filtrados

Etiquetado de estructuras gramaticales:

In [ ]:
pos_tags_filtrados = texto_analizado['pos_tags']

In [ ]:
pos_tags_filtrados

### Análisis de texto


Detectar presencia de palabras de productos lematizadas dentro de dataframe

In [ ]:
df_frutas_verduras['producto_tokens_lemmas'] = df_frutas_verduras['Producto'].apply(lambda x: procesar_texto(x)['lemmas'])

In [ ]:
df_frutas_verduras['producto_tokens_lemmas']

In [ ]:
dict_count_coincidences = {}

In [ ]:
for index, row in df_frutas_verduras.iterrows():
    for token_producto in row['producto_tokens_lemmas']:
        if token_producto in lemmas_filtrados:
            if token_producto not in dict_count_coincidences:
                dict_count_coincidences[index] = 1
            else:
                dict_count_coincidences[index] += 1

In [ ]:
series_ordered_count_coincidences = pd.Series(dict_count_coincidences)

Análisis de Sentimiento: Determinar la emoción o tono del texto.


In [ ]:
#para futuras versiones

Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


In [ ]:
#para futuras versiones

## 3. Gestión del Diálogo

___

### Módulo de Gestión de Estado:
Llevar un registro del contexto y estado del diálogo para mantener conversaciones coherentes.


In [ ]:
columns_consultas_dict = ['id_consulta', 'id_cliente', 'formato_consulta', 'transcripción_audio', 'entrada_texto', 'productos_lemas_filtrados', 'dict_producto_indice_considencias']

In [ ]:
#consulta_dict['id_consulta'] = agregaridfran
#consulta_dict['id_cliente'] = agregaridclientefran
#consulta_dict['formato_consulta'] = 
#consulta_dict['transcripción_audio'] = 
consulta_dict['entrada_texto'] = texto_prueba
consulta_dict['entrada_lemas_filtrados'] = lemmas_filtrados
consulta_dict['dict_producto_indice_considencias'] = dict_count_coincidences

In [ ]:
consulta_dict

### Motor de Respuesta:


Generación de Respuestas: Utilizar modelos generativos (como GPT-3) o respuestas predefinidas según las intenciones y entidades detectadas.


In [ ]:
# Asegúrate de que series_ordered_count_coincidences es una Serie ordenada
series_ordered_count_coincidences = series_ordered_count_coincidences.sort_values(ascending=False)

# Inicialización de la recomendación
recomendacion = 'Los productos recomendados en base a su consulta y caracteristicas mensionadas son:\n'

# Lista para almacenar las recomendaciones
lista_string_reco_supers_prods = []

# Iterar sobre los índices de las coincidencias ordenadas
for count, x in enumerate(series_ordered_count_coincidences.index, start=1):
    # Extraer los detalles del producto usando el índice
    producto, supermercado, precio = df_frutas_verduras.loc[x, ['Producto', 'Supermercado', 'Precio']]
    
    # Añadir la recomendación a la lista
    lista_string_reco_supers_prods.append(f'En el lugar {count}: {producto} en {supermercado} a {precio}\n')

# Concatenar todas las recomendaciones en un solo string
recomendacion += ''.join(lista_string_reco_supers_prods)

print(recomendacion)


Selección de Respuestas: Elegir la mejor respuesta entre varias opciones generadas.


In [ ]:
#Para proximas versiones

### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario y el contexto actual.

In [ ]:
#Para proximas versiones

## 4. Texto a voz

___

### Conversión de Texto a Voz (TTS): Utilizar servicios como Google Text-to-Speech, Amazon Polly, o frameworks como Tacotron para convertir el texto generado en voz.


In [ ]:
recomendacion = common_functions.limpiar_signo_peso(recomendacion)

In [ ]:
tts = gTTS(text=recomendacion, lang='es')

In [ ]:
tts.save("../datos/procesados/proc_habla/output.mp3")